In [2]:
!pip install flask

  Using cached Flask-2.0.2-py3-none-any.whl (95 kB)
  Using cached itsdangerous-2.0.1-py3-none-any.whl (18 kB)
  Using cached MarkupSafe-2.0.1-cp39-cp39-win_amd64.whl (14 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 1.1.1
    Uninstalling MarkupSafe-1.1.1:
      Successfully uninstalled MarkupSafe-1.1.1
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cookiecutter 1.7.2 requires Jinja2<3.0.0, but you have jinja2 3.0.3 which is incompatible.
cookiecutter 1.7.2 requires MarkupSafe<2.0.0, but you have markupsafe 2.0.1 which is incompatible.
You should consider upgrading via the 'D:\Users\Lenovo\anaconda3\envs\r4-base\python.exe -m pip install --upgrade pip' command.


In [4]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec  3 19:12:06 2021

@author: Lenovo
"""

import os
from flask import Flask, request, render_template, render_template_string
from werkzeug.utils import secure_filename
import pandas as pd
import joblib
import numpy as np
!pip install xgboost
from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer

model = joblib.load('xgb.joblib')


app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = '.'


@app.route('/')
def index():
    return render_template_string('''<!DOCTYPE html>

<html>

<head>
    <title>Upload</title>
</head>

<body>

<form id="my_form" method="POST" action="/upload" enctype="multipart/form-data">
    <input type="file" name="my_file" />
    <input type="submit" id="my_button" value="Send" />
</form>

<div id="result"></div>

<script>
var my_form = document.getElementById("my_form");
var my_file = document.getElementById("my_file");
var my_button = document.getElementById("my_button");
var result = document.getElementById("result");

my_button.onclick = function(event){

    var formData = new FormData(my_form);
    console.log(formData)
    formData.append('my_file', my_file);
    console.log(formData)

    var xhr = new XMLHttpRequest();
    // Add any event handlers here...
    xhr.open('POST', '/upload', true);

    xhr.addEventListener('load', function(e) {
        // HTTP status message
        //console.log(xhr.status);
        // request.response will hold the response from the server
        //console.log(xhr.response);
        result.innerHTML = xhr.response;
    });

    xhr.send(formData);

    event.preventDefault(); 
};
</script>

</body>

</html>''')


@app.route('/upload', methods=['POST'])
def upload():

    #if request.method == 'POST': # no need if `methods=['POST']` 
    #if request.is_xhr():  # `is_xhr()` doesn't exist any more
    
    # get file     
    #print(request.files)
    temp_file = request.files.get('my_file')  #  `<input ... name="my_file">`
    print(temp_file.filename)

    # save it in 
    save_name = secure_filename(temp_file.filename)
    full_path = os.path.join(app.config['UPLOAD_FOLDER'], save_name)
    temp_file.save(full_path)

    
    df = pd.read_csv(full_path).drop('price', axis=1).set_index('id')
    df.drop(['date', 'yr_built'], axis=1, inplace=True)
    df['predicted'] = np.round(model.predict(df), -2)
    
    
    
    '''
    data = pd.DataFrame({
        'Text': ['abc', 'def', 'ghi'],
        'Number': [123, 456, 789],
    })
    '''
    print(df)

    # convert dataFrame to HTML 
    table = df.to_html() 
    #table = data.to_html(classes="my_table", header=True) 

    # send generated HTML
    return table

    # or

    # send template with embed HTML. It  needs `|safe` to display HTML correctly
    #return render_template_string('''DataFrame:</br>{{ table|safe }}</br>''', table=table)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)


You should consider upgrading via the 'D:\Users\Lenovo\anaconda3\envs\r4-base\python.exe -m pip install --upgrade pip' command.


 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Dec/2021 22:19:45] "GET / HTTP/1.1" 200 -


sample.csv


...

            bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
id                                                                           
9346920070         3       1.75         1930     10183     1.0           0   
2976800749         4       2.00         1460      7254     1.0           0   
1771110550         3       1.00         1330      9540     1.0           0   
9407110710         3       1.75         1510      8400     1.0           0   
5132000140         6       1.00         1370      5080     1.5           0   
6719600030         5       2.75         2940      5225     2.0           0   
1250202115         3       1.75         1670      5100     1.0           0   
7283900045         3       2.50         2160      6288     2.0           0   
1871400585         2       1.00         1020     13647     1.0           0   
3578400950         3       1.75         1540     13002     1.0           0   

            view  condition  grade  sqft_above  ...  yr_renovat

127.0.0.1 - - [03/Dec/2021 23:37:33] "GET / HTTP/1.1" 200 -


sample.csv


127.0.0.1 - - [03/Dec/2021 23:37:55] "POST /upload HTTP/1.1" 200 -


            bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
id                                                                           
9346920070         3       1.75         1930     10183     1.0           0   
2976800749         4       2.00         1460      7254     1.0           0   
1771110550         3       1.00         1330      9540     1.0           0   
9407110710         3       1.75         1510      8400     1.0           0   
5132000140         6       1.00         1370      5080     1.5           0   
6719600030         5       2.75         2940      5225     2.0           0   
1250202115         3       1.75         1670      5100     1.0           0   
7283900045         3       2.50         2160      6288     2.0           0   
1871400585         2       1.00         1020     13647     1.0           0   
3578400950         3       1.75         1540     13002     1.0           0   

            view  condition  grade  sqft_above  ...  yr_renovat